# 🕸️ 第一章 练习 2: 因果图与 DAG

---

## 从「看图说话」到「看图断因果」

在上一个练习中，我们学习了潜在结果框架。但是，当你遇到这样的问题时：

> "这个变量应该控制吗？控制了会怎样？不控制又会怎样？"

单纯靠数学公式就不够直观了。这时候，**因果图 (Causal Graph)** 就派上用场了！

### 一个有趣的例子 🏥

你是一名流行病学家，在研究「运动」对「健康」的影响。你发现：

- 经常运动的人更健康 ✓
- 但经常运动的人收入也更高...
- 而高收入的人本来就更健康（更好的医疗、饮食）...

所以运动到底有没有用？收入是怎么回事？应该怎么分析？

用因果图，这个问题一画就清楚了！

---

## 📚 本节学习目标

1. 理解因果图的基本概念
2. 识别三种核心结构：混淆、中介、碰撞
3. 理解后门路径和 d-分离
4. 掌握「应该控制哪些变量」的决策方法

In [ ]:
# 导入必要的库
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from typing import List, Set, Tuple

# 设置
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'SimHei']
plt.rcParams['axes.unicode_minus'] = False
sns.set_style('whitegrid')

print("✅ 环境准备完毕！")

---

## 🌟 Part 1: 什么是因果图 (DAG)？

### DAG = Directed Acyclic Graph（有向无环图）

- **有向 (Directed)**: 箭头指向因果的方向（原因 → 结果）
- **无环 (Acyclic)**: 不能有循环（A → B → C → A 是不允许的）
- **图 (Graph)**: 用节点和边表示变量之间的关系

### 最简单的因果图

```
T → Y
```

这表示：T **导致** Y（T 是原因，Y 是结果）

### 经典的混淆结构

```
    X
   ↙ ↘
  T   Y
   ↘ ↗
```

等价于：
- X → T（X 影响 T）
- X → Y（X 影响 Y）  
- T → Y（T 影响 Y）

这里 X 就是**混淆变量 (Confounder)**！

### 🎨 用代码画因果图

让我们用一个简单的函数来可视化因果图：

In [ ]:
def draw_dag(edges, title="因果图 (DAG)", highlight_path=None):
    """
    画一个简单的因果图
    
    Args:
        edges: 边的列表，如 [("X", "T"), ("X", "Y"), ("T", "Y")]
        title: 图的标题
        highlight_path: 要高亮的路径
    """
    try:
        import networkx as nx
        
        G = nx.DiGraph()
        G.add_edges_from(edges)
        
        # 设置节点位置
        pos = nx.spring_layout(G, seed=42)
        
        plt.figure(figsize=(8, 6))
        
        # 画边
        nx.draw_networkx_edges(G, pos, edge_color='gray', 
                               arrows=True, arrowsize=20,
                               connectionstyle="arc3,rad=0.1")
        
        # 如果有高亮路径
        if highlight_path:
            highlight_edges = [(highlight_path[i], highlight_path[i+1]) 
                              for i in range(len(highlight_path)-1)]
            nx.draw_networkx_edges(G, pos, edgelist=highlight_edges,
                                   edge_color='red', width=2,
                                   arrows=True, arrowsize=25)
        
        # 画节点
        nx.draw_networkx_nodes(G, pos, node_color='lightblue', 
                               node_size=2000, alpha=0.9)
        nx.draw_networkx_labels(G, pos, font_size=14, font_weight='bold')
        
        plt.title(title, fontsize=14)
        plt.axis('off')
        plt.tight_layout()
        plt.show()
        
    except ImportError:
        print("需要安装 networkx: pip install networkx")
        print(f"\n边: {edges}")

# 画一个简单的混淆结构
confounding_edges = [("X", "T"), ("X", "Y"), ("T", "Y")]
draw_dag(confounding_edges, "经典混淆结构: X 是混淆变量")

---

## 🔍 Part 2: 三种核心因果结构

因果图中有三种最基本的结构，理解它们是掌握因果推断的关键！

### 结构 1: 叉子 (Fork) - 混淆

```
T ← X → Y
```

**特点**: X 同时影响 T 和 Y

**例子**: 
- X = 年龄
- T = 是否喝咖啡
- Y = 是否有心脏病

年轻人喝咖啡更多，但年轻人心脏病也更少。如果不控制年龄，会错误地认为咖啡预防心脏病！

**规则**: 需要控制 X 来阻断混淆

---

### 结构 2: 链条 (Chain) - 中介

```
T → M → Y
```

**特点**: T 通过 M 影响 Y

**例子**:
- T = 受教育程度
- M = 收入
- Y = 健康状况

教育通过提高收入来改善健康。M 是中介变量。

**规则**: 
- 如果想估计 T 的**总效应**，不要控制 M
- 如果想估计 T 的**直接效应**，需要控制 M

---

### 结构 3: 碰撞 (Collider) - 碰撞变量

```
T → C ← Y
```

**特点**: T 和 Y 都影响 C

**例子** (经典！):
- T = 演技好
- Y = 颜值高
- C = 成为明星

成为明星需要演技**或**颜值（至少一个）。在明星群体中，演技好和颜值高会呈现负相关——因为只靠颜值的人演技可能不好，只靠演技的人可能不是靠脸吃饭。

**规则**: 千万不要控制碰撞变量 C！否则会创造虚假关联

In [ ]:
# 画出三种结构
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

structures = [
    ([("X", "T"), ("X", "Y")], "Fork (叉子/混淆)\nT ← X → Y\n控制 X ✓"),
    ([("T", "M"), ("M", "Y")], "Chain (链条/中介)\nT → M → Y\n通常不控制 M"),
    ([("T", "C"), ("Y", "C")], "Collider (碰撞)\nT → C ← Y\n绝对不控制 C ✗")
]

for idx, (edges, title) in enumerate(structures):
    ax = axes[idx]
    try:
        import networkx as nx
        G = nx.DiGraph()
        G.add_edges_from(edges)
        pos = nx.spring_layout(G, seed=42)
        
        nx.draw_networkx_edges(G, pos, ax=ax, edge_color='gray', 
                               arrows=True, arrowsize=20)
        nx.draw_networkx_nodes(G, pos, ax=ax, node_color='lightblue', 
                               node_size=1500, alpha=0.9)
        nx.draw_networkx_labels(G, pos, ax=ax, font_size=12, font_weight='bold')
        ax.set_title(title, fontsize=11)
        ax.axis('off')
    except:
        ax.text(0.5, 0.5, f"边: {edges}\n{title}", ha='center', va='center')
        ax.axis('off')

plt.tight_layout()
plt.show()

### 🧪 练习: 识别因果结构

In [ ]:
def identify_structure(edges: List[Tuple[str, str]], node: str) -> str:
    """
    根据 DAG 边，识别给定节点的因果结构类型
    
    结构类型:
    - "confounder": 混淆变量 (出度 >= 2，像叉子的中心)
    - "mediator": 中介变量 (入度 >= 1 且 出度 >= 1，在链条中间)
    - "collider": 碰撞变量 (入度 >= 2，像碰撞的中心)
    
    Args:
        edges: 边列表，如 [("X", "T"), ("X", "Y"), ("T", "Y")]
        node: 要识别的节点
    
    Returns:
        结构类型字符串
    """
    # === 你的代码开始 ===
    # 提示 1: 计算入度 (有多少条边指向这个节点)
    in_degree = None  # 你的代码
    
    # 提示 2: 计算出度 (这个节点有多少条边指出去)
    out_degree = None  # 你的代码
    
    # 提示 3: 根据入度和出度判断结构类型
    # - 碰撞变量 (collider): 入度 >= 2, 出度 = 0 (多个箭头指向它)
    # - 混淆变量 (confounder): 出度 >= 2 (它指向多个节点)
    # - 中介变量 (mediator): 入度 >= 1 且 出度 >= 1 (在链条中间)
    
    pass  # 删除此行，填入你的代码
    # === 你的代码结束 ===

<details>
<summary>📝 参考答案（点击展开）</summary>

```python
def identify_structure(edges: List[Tuple[str, str]], node: str) -> str:
    """根据 DAG 边，识别给定节点的因果结构类型"""
    # 计算入度和出度
    in_degree = sum(1 for edge in edges if edge[1] == node)
    out_degree = sum(1 for edge in edges if edge[0] == node)
    
    # 根据入度和出度判断结构类型
    if in_degree >= 2 and out_degree == 0:
        return "collider"  # 碰撞变量：多个箭头指向它
    elif out_degree >= 2:
        return "confounder"  # 混淆变量：它指向多个节点
    elif in_degree >= 1 and out_degree >= 1:
        return "mediator"  # 中介变量：在链条中间
    else:
        return "other"
```

**解释**：
- **Collider（碰撞）**: in_degree ≥ 2 且 out_degree = 0，如 T → C ← Y
  - 特点：多个箭头指向它，像碰撞点
  - 规则：不能控制，否则会打开路径
  
- **Confounder（混淆）**: out_degree ≥ 2，如 X → T 和 X → Y
  - 特点：它指向多个节点，是「叉子」的中心
  - 规则：必须控制，阻断后门路径
  
- **Mediator（中介）**: in_degree ≥ 1 且 out_degree ≥ 1，如 T → M → Y
  - 特点：在因果链条中间
  - 规则：估计总效应时不控制，估计直接效应时控制
</details>

In [ ]:
# 测试你的代码
test_edges = [("X", "T"), ("X", "Y"), ("T", "Y")]

print("DAG: X → T, X → Y, T → Y")
print("\n识别每个节点的角色:")

for node in ["X", "T", "Y"]:
    structure = identify_structure(test_edges, node)
    if structure:
        print(f"   节点 {node}: {structure}")
    else:
        print(f"   节点 {node}: [未完成]")

---

## 🛤️ Part 3: 路径与后门准则

### 什么是「路径」？

在因果图中，从 T 到 Y 可能有多条路径：

```
    X
   ↙ ↘
  T → Y
```

从 T 到 Y 有两条路径：
1. **直接路径 (因果路径)**: T → Y ✅
2. **后门路径**: T ← X → Y ⚠️

### 什么是「后门路径」？

后门路径是从 T 到 Y 的路径中，第一个箭头**指向 T** 的路径。

- T → Y 不是后门（箭头从 T 出发）
- T ← X → Y 是后门（第一个箭头指向 T）

**后门路径会造成混淆偏差！我们需要把它阻断。**

### 后门准则 (Backdoor Criterion)

要无偏地估计 T 对 Y 的因果效应，需要找到一组变量 Z 满足：

1. Z 阻断了所有从 T 到 Y 的后门路径
2. Z 不包含 T 的任何后代

In [ ]:
# 📖 示范代码: 找出 DAG 中的所有路径

def find_all_paths(edges: List[Tuple[str, str]], 
                   start: str, 
                   end: str,
                   ignore_direction: bool = True) -> List[List[str]]:
    """
    找出 DAG 中从 start 到 end 的所有路径
    
    Args:
        edges: 边列表
        start: 起始节点
        end: 终止节点
        ignore_direction: 是否忽略边的方向（找后门路径时需要忽略）
    
    Returns:
        路径列表，每条路径是节点序列
    """
    # 构建邻接表
    adj = {}
    for u, v in edges:
        if u not in adj:
            adj[u] = []
        adj[u].append(v)
        
        if ignore_direction:  # 如果忽略方向，也添加反向边
            if v not in adj:
                adj[v] = []
            adj[v].append(u)
    
    paths = []
    
    # 使用 DFS 找出所有路径
    def dfs(current, path, visited):
        """
        深度优先搜索找路径
        
        current: 当前节点
        path: 当前路径
        visited: 已访问节点集合
        """
        # 如果到达终点，保存路径
        if current == end:
            paths.append(path.copy())
            return
        
        # 遍历邻居
        for neighbor in adj.get(current, []):
            if neighbor not in visited:
                # 将邻居加入已访问，递归调用，然后移除（回溯）
                visited.add(neighbor)
                path.append(neighbor)
                dfs(neighbor, path, visited)
                path.pop()
                visited.remove(neighbor)
    
    dfs(start, [start], {start})
    return paths

<details>
<summary>📝 参考答案（点击展开）</summary>

```python
def identify_backdoor_paths(edges: List[Tuple[str, str]], 
                            treatment: str, 
                            outcome: str) -> List[List[str]]:
    """识别从 treatment 到 outcome 的后门路径"""
    all_paths = find_all_paths(edges, treatment, outcome, ignore_direction=True)
    
    backdoor_paths = []
    for path in all_paths:
        if len(path) < 2:
            continue
        
        # 检查是否是后门路径
        # 后门路径的第一条边指向 treatment
        first_step = path[1]  # 路径上的第二个节点
        
        # 检查边 first_step → treatment 是否存在
        is_backdoor = (first_step, treatment) in edges
        
        if is_backdoor:
            backdoor_paths.append(path)
    
    return backdoor_paths
```

**解释**：
- **后门路径定义**: 从 T 到 Y 的路径，第一条边指向 T（箭头朝向 T）
  
- **识别方法**:
  1. 找到所有从 T 到 Y 的路径（忽略方向）
  2. 对每条路径，检查第一步
  3. 如果第一步是 path[1] → T（即 (path[1], T) 在边集中），则是后门路径
  
- **例子**: 
  ```
  DAG: X → T, X → Y, T → Y
  路径: T ← X → Y (即 [T, X, Y])
  第一步: X → T (在边集中) ✓ 是后门路径
  ```
  
- **为什么重要**: 后门路径会引入混淆偏差，必须阻断它们
</details>

In [ ]:
# 📖 示范代码: 识别后门路径

def identify_backdoor_paths(edges: List[Tuple[str, str]], 
                            treatment: str, 
                            outcome: str) -> List[List[str]]:
    """
    识别从 treatment 到 outcome 的后门路径
    
    后门路径: 从 T 到 Y 的路径，其中第一条边指向 T
    
    Returns:
        后门路径列表
    """
    all_paths = find_all_paths(edges, treatment, outcome, ignore_direction=True)
    
    backdoor_paths = []
    for path in all_paths:
        if len(path) < 2:
            continue
        
        # 检查是否是后门路径
        # 后门路径的第一条边指向 treatment
        # 即检查 (path[1], treatment) 是否在 edges 中
        
        first_step = path[1]  # 路径上的第二个节点
        
        # 检查边 first_step → treatment 是否存在
        is_backdoor = (first_step, treatment) in edges
        
        if is_backdoor:
            backdoor_paths.append(path)
    
    return backdoor_paths

<details>
<summary>📝 参考答案（点击展开）</summary>

```python
def is_valid_adjustment_set(edges: List[Tuple[str, str]], 
                            treatment: str, 
                            outcome: str,
                            adjustment_set: Set[str]) -> bool:
    """检查给定的调整集是否有效（满足后门准则）"""
    # 步骤 1: 找出 treatment 的所有后代
    def find_descendants(node):
        descendants = set()
        to_visit = [n for (s, n) in edges if s == node]
        while to_visit:
            current = to_visit.pop()
            if current not in descendants:
                descendants.add(current)
                to_visit.extend([n for (s, n) in edges if s == current])
        return descendants
    
    descendants_of_T = find_descendants(treatment)
    
    # 检查调整集是否包含 treatment 的后代
    contains_descendant = bool(adjustment_set & descendants_of_T)
    
    if contains_descendant:
        return False  # 违反条件 2
    
    # 步骤 2: 找出所有后门路径
    backdoor_paths = identify_backdoor_paths(edges, treatment, outcome)
    
    # 检查调整集是否阻断所有后门路径
    for path in backdoor_paths:
        path_nodes = set(path) - {treatment, outcome}
        # 检查 adjustment_set 是否与 path_nodes 有交集
        is_blocked = bool(adjustment_set & path_nodes)
        
        if not is_blocked:
            return False  # 有后门路径没被阻断
    
    return True
```

**解释**：
- **后门准则两个条件**:
  1. 调整集阻断所有后门路径
  2. 调整集不包含 treatment 的后代
  
- **条件 1 验证**:
  - 对每条后门路径，检查调整集是否包含路径上至少一个节点
  - 使用集合交集运算：`adjustment_set & path_nodes`
  - 如果交集非空，说明路径被阻断
  
- **条件 2 验证**:
  - 找出 T 的所有后代（递归搜索）
  - 检查调整集是否与后代集合有交集
  - 如果有交集，违反条件（可能阻断因果路径）
  
- **为什么不能控制后代**: 
  - T → M → Y 中，控制 M 会阻断因果路径
  - 我们想估计 T 的总效应，不能阻断 T 的作用机制
</details>

In [ ]:
# 测试后门路径识别
test_edges = [("X", "T"), ("X", "Y"), ("T", "Y")]

print("DAG: X → T, X → Y, T → Y")
print("\n寻找从 T 到 Y 的路径...")

all_paths = find_all_paths(test_edges, "T", "Y", ignore_direction=True)
print(f"\n所有路径: {all_paths}")

backdoor = identify_backdoor_paths(test_edges, "T", "Y")
print(f"后门路径: {backdoor}")

if backdoor:
    print(f"\n✅ 需要控制的变量: {set(sum(backdoor, [])) - {'T', 'Y'}}")

<details>
<summary>📝 参考答案（点击展开）</summary>

```python
def simulate_confounding_dag(n: int = 2000, seed: int = 42):
    """模拟经典混淆 DAG 的数据"""
    np.random.seed(seed)
    
    # 生成混淆变量 X
    X = np.random.randn(n)
    
    # 生成处理 T（受 X 影响）
    T = (X + np.random.randn(n) > 0).astype(int)
    
    # 生成结果 Y（受 T 和 X 影响）
    Y = 1 + 2 * T + 1.5 * X + np.random.randn(n) * 0.5
    
    df = pd.DataFrame({'X': X, 'T': T, 'Y': Y})
    return df
```

**解释**：
- **DAG 结构**: X → T, X → Y, T → Y（经典混淆）
  
- **数据生成过程**:
  - X ~ N(0, 1): 标准正态分布的混淆变量
  - T = 1[X + ε > 0]: X 越大，T=1 的概率越高
  - Y = 1 + 2T + 1.5X + ε: 真实 ATE = 2
  
- **混淆机制**:
  - X 同时影响 T（系数隐含在 P(T=1|X) 中）
  - X 同时影响 Y（系数 = 1.5）
  - 不控制 X 会导致偏差
  
- **验证**: 
  - 朴素估计会偏高（因为 T=1 的人 X 更大，Y 也更大）
  - 控制 X 后估计应该接近 2
</details>

In [ ]:
# 📖 示范代码: 检查调整集是否满足后门准则

def is_valid_adjustment_set(edges: List[Tuple[str, str]], 
                            treatment: str, 
                            outcome: str,
                            adjustment_set: Set[str]) -> bool:
    """
    检查给定的调整集是否有效（满足后门准则）
    
    后门准则要求调整集:
    1. 阻断所有后门路径
    2. 不包含 treatment 的后代
    
    Returns:
        True 如果调整集有效
    """
    # 步骤 1: 找出 treatment 的所有后代
    def find_descendants(node):
        """找出一个节点的所有后代"""
        descendants = set()
        to_visit = [n for (s, n) in edges if s == node]
        while to_visit:
            current = to_visit.pop()
            if current not in descendants:
                descendants.add(current)
                to_visit.extend([n for (s, n) in edges if s == current])
        return descendants
    
    descendants_of_T = find_descendants(treatment)
    
    # 检查调整集是否包含 treatment 的后代
    contains_descendant = bool(adjustment_set & descendants_of_T)
    
    if contains_descendant:
        return False  # 违反条件 2
    
    # 步骤 2: 找出所有后门路径
    backdoor_paths = identify_backdoor_paths(edges, treatment, outcome)
    
    # 检查调整集是否阻断所有后门路径
    # 一条路径被阻断当且仅当调整集包含路径上的至少一个节点（除了 T 和 Y）
    for path in backdoor_paths:
        path_nodes = set(path) - {treatment, outcome}
        # 检查 adjustment_set 是否与 path_nodes 有交集
        is_blocked = bool(adjustment_set & path_nodes)
        
        if not is_blocked:
            return False  # 有后门路径没被阻断
    
    return True

In [ ]:
# 测试调整集
test_edges = [("X", "T"), ("X", "Y"), ("T", "Y")]

print("DAG: X → T, X → Y, T → Y")
print("\n检验不同的调整集:")

adjustment_sets = [set(), {"X"}, {"Y"}]
for adj_set in adjustment_sets:
    result = is_valid_adjustment_set(test_edges, "T", "Y", adj_set)
    status = "✅ 有效" if result else "❌ 无效"
    print(f"   调整集 {adj_set if adj_set else '{}'}: {status}")

<details>
<summary>📝 参考答案（点击展开）</summary>

```python
def simulate_collider_bias(n: int = 2000, seed: int = 42):
    """模拟碰撞偏差"""
    np.random.seed(seed)
    
    # 生成演技 T（独立）
    T = np.random.uniform(0, 10, n)
    
    # 生成颜值 Y（独立）
    Y = np.random.uniform(0, 10, n)
    
    # 生成碰撞变量 C（是否成为明星）
    C = (T + Y > 12).astype(int)
    
    df = pd.DataFrame({'演技': T, '颜值': Y, '明星': C})
    
    # 计算相关性
    overall_corr = np.corrcoef(T, Y)[0, 1]
    
    # 只看明星的相关性
    stars = df[df['明星'] == 1]
    conditional_corr = np.corrcoef(stars['演技'], stars['颜值'])[0, 1] if len(stars) > 10 else 0
    
    return df, overall_corr, conditional_corr
```

**解释**：
- **DAG 结构**: T → C ← Y（碰撞结构）
  - T（演技）和 Y（颜值）本来是**独立的**
  - C（是否成为明星）是碰撞变量
  
- **碰撞条件**: T + Y > 12
  - 需要演技和颜值的总和足够高才能成为明星
  - 可以演技很高颜值一般，或颜值很高演技一般
  - 这创造了「替代」关系
  
- **好莱坞悖论**:
  - 总人口：演技和颜值相关性 ≈ 0（独立）
  - 明星群体：演技和颜值呈现**负相关**
  - 原因：在明星中，高演技的人可能颜值一般（靠演技进来的）
  
- **启示**: 控制碰撞变量会创造虚假关联！
</details>

In [ ]:
# 📖 示范代码: 模拟经典混淆 DAG

def simulate_confounding_dag(n: int = 2000, seed: int = 42):
    """
    模拟经典混淆 DAG 的数据
    
    DAG: X → T, X → Y, T → Y
    
    数据生成过程:
    - X ~ N(0, 1)
    - T = 1 if X + noise > 0 else 0
    - Y = 1 + 2*T + 1.5*X + noise
    
    真实 ATE = 2
    """
    np.random.seed(seed)
    
    # 生成混淆变量 X
    X = np.random.randn(n)
    
    # 生成处理 T（受 X 影响）
    # T = 1 当 X + noise > 0
    T = (X + np.random.randn(n) > 0).astype(int)
    
    # 生成结果 Y（受 T 和 X 影响）
    # Y = 1 + 2*T + 1.5*X + noise
    Y = 1 + 2 * T + 1.5 * X + np.random.randn(n) * 0.5
    
    df = pd.DataFrame({'X': X, 'T': T, 'Y': Y})
    
    return df

In [ ]:
def estimate_ate_with_adjustment(df: pd.DataFrame, 
                                  adjustment_vars: List[str]) -> float:
    """
    使用线性回归进行调整估计
    
    Y = b0 + b1*T + b2*X1 + b3*X2 + ...
    b1 就是调整后的 ATE 估计
    """
    from sklearn.linear_model import LinearRegression
    
    # 构建特征矩阵
    features = ['T'] + adjustment_vars
    X = df[features].values
    y = df['Y'].values
    
    # 拟合线性回归
    model = LinearRegression()
    model.fit(X, y)
    
    # T 的系数就是调整后的 ATE
    return model.coef_[0]

In [ ]:
# 运行模拟
df = simulate_confounding_dag(n=5000)

if df is not None and df['X'] is not None:
    true_ate = 2.0
    
    # 朴素估计（不控制任何变量）
    naive_ate = df[df['T']==1]['Y'].mean() - df[df['T']==0]['Y'].mean()
    
    # 调整估计（控制 X）
    adjusted_ate = estimate_ate_with_adjustment(df, ['X'])
    
    print("🔬 混淆 DAG 模拟结果:")
    print("=" * 50)
    print(f"真实 ATE: {true_ate:.4f}")
    print(f"\n朴素估计（不控制 X）: {naive_ate:.4f}")
    print(f"   偏差: {naive_ate - true_ate:+.4f} {'⚠️ 有偏！' if abs(naive_ate - true_ate) > 0.1 else ''}")
    print(f"\n调整估计（控制 X）: {adjusted_ate:.4f}")
    print(f"   偏差: {adjusted_ate - true_ate:+.4f} {'✅ 很准！' if abs(adjusted_ate - true_ate) < 0.1 else ''}")
else:
    print("❌ 请完成 simulate_confounding_dag 函数！")

In [ ]:
# 📖 示范代码: 模拟碰撞偏差（好莱坞悖论）

def simulate_collider_bias(n: int = 2000, seed: int = 42):
    """
    模拟碰撞偏差
    
    DAG: T → C ← Y（T 和 Y 本来是独立的！）
    
    场景: 好莱坞悖论
    - T = 演技（0到10分）
    - Y = 颜值（0到10分）
    - C = 是否成为明星（需要演技+颜值 > 12）
    
    演技和颜值是独立的！但在明星群体中会呈现负相关。
    """
    np.random.seed(seed)
    
    # 生成演技 T（独立）
    T = np.random.uniform(0, 10, n)
    
    # 生成颜值 Y（独立）
    Y = np.random.uniform(0, 10, n)
    
    # 生成碰撞变量 C（是否成为明星）
    # 当 T + Y > 12 时成为明星
    C = (T + Y > 12).astype(int)
    
    df = pd.DataFrame({'演技': T, '颜值': Y, '明星': C})
    
    # 计算相关性
    overall_corr = np.corrcoef(T, Y)[0, 1]
    
    # 只看明星的相关性
    stars = df[df['明星'] == 1]
    conditional_corr = np.corrcoef(stars['演技'], stars['颜值'])[0, 1] if len(stars) > 10 else 0
    
    return df, overall_corr, conditional_corr

In [ ]:
# 运行碰撞偏差模拟
df_collider, overall, conditional = simulate_collider_bias(n=5000)

if overall is not None:
    print("🎬 好莱坞悖论模拟:")
    print("=" * 50)
    print(f"总人口中演技和颜值的相关性: {overall:.4f}")
    print(f"   解读: {'几乎不相关 ✅' if abs(overall) < 0.1 else '有相关性'}")
    print(f"\n只看明星时演技和颜值的相关性: {conditional:.4f}")
    print(f"   解读: {'强负相关 ⚠️' if conditional < -0.3 else '有负相关'}")
    
    print(f"\n明星占总人口: {df_collider['明星'].mean()*100:.1f}%")
else:
    print("❌ 请完成 simulate_collider_bias 函数！")

---

## 🎯 面试题模拟

### 概念题

**Q1: 什么是因果图（DAG）？它与传统的统计图（如相关系数矩阵）有什么区别？**

<details>
<summary>答案</summary>

**因果图（DAG）**:
- Directed Acyclic Graph（有向无环图）
- 节点表示变量，边表示因果关系（箭头方向表示因果方向）
- 编码了变量之间的因果假设，而非仅仅是统计关联

**与相关系数矩阵的区别**:

| 特性 | 因果图 | 相关系数矩阵 |
|-----|--------|------------|
| 方向性 | 有方向（A → B ≠ B → A） | 无方向（对称） |
| 因果性 | 编码因果关系 | 仅编码相关性 |
| 条件独立 | d-分离→条件独立 | 无法推断 |
| 干预 | 可预测干预效果 | 无法预测 |

**例子**:
- 相关系数：冰淇淋销量和溺水人数正相关（r = 0.7）
- 因果图：温度 → 冰淇淋销量，温度 → 游泳人数 → 溺水
- 结论：禁售冰淇淋不会减少溺水（因果图正确，相关性误导）

</details>

---

**Q2: 解释后门准则（Backdoor Criterion）的含义，并举例说明什么样的变量集合满足后门准则。**

<details>
<summary>答案</summary>

**后门准则定义**:

变量集合 Z 满足相对于 (T, Y) 的后门准则，需满足：
1. Z 阻断所有从 T 到 Y 的后门路径
2. Z 不包含 T 的任何后代

**直觉**: Z 关闭了「混淆的门」，但保留了「因果的路径」。

**例子 1: 简单混淆**
```
DAG: X → T, X → Y, T → Y
后门路径: T ← X → Y
调整集: {X} ✓ 满足
- X 阻断唯一后门路径
- X 不是 T 的后代
```

**例子 2: 中介变量**
```
DAG: T → M → Y, X → T, X → Y
后门路径: T ← X → Y
调整集: {M} ✗ 不满足
- M 是 T 的后代（违反条件2）
- 控制 M 会阻断因果路径 T → M → Y
正确调整集: {X}
```

**例子 3: 碰撞变量**
```
DAG: T → C ← Y, T → Y
调整集: {C} ✗ 不满足
- C 是碰撞变量，本来阻断，控制后反而打开路径
- 会引入虚假关联
正确调整集: {} (空集，不需要控制任何变量)
```

**实践意义**: 后门准则告诉我们「应该控制哪些变量」来识别因果效应。

</details>

---

**Q3: 为什么控制碰撞变量会引入偏差？用好莱坞悖论或伯克利性别歧视案解释。**

<details>
<summary>答案</summary>

**碰撞变量的特性**:
- 结构：T → C ← Y
- T 和 Y 本来独立，但都影响 C
- 控制 C（条件于 C=某值）会打开 T 和 Y 之间的虚假关联

**好莱坞悖论例子**:

DAG: 演技 → 成为明星 ← 颜值

1. **总人口**: 演技和颜值独立（相关性≈0）
2. **只看明星**: 演技和颜值负相关（相关性<0）
3. **原因**: 
   - 成为明星需要演技+颜值足够高
   - 在明星中，高演技的人可能颜值一般（靠演技进来）
   - 在明星中，高颜值的人可能演技一般（靠脸进来）
   - 这是「替代效应」

**数学直觉**:

条件概率的贝叶斯公式：
$$P(T=1 \mid C=1, Y=0) > P(T=1 \mid C=1, Y=1)$$

解释：
- 如果已知某人是明星（C=1）且颜值不高（Y=0）
- 那么他演技高（T=1）的概率更大（否则怎么成为明星？）
- 这就是负相关

**实践警示**:
- 不要控制碰撞变量（如「只看成功者」「只看存活者」）
- 存活者偏差（Survivorship Bias）就是碰撞偏差的一种

</details>

---

### 编程题

**Q4: 给定一个 DAG，请实现函数找出所有满足后门准则的最小调整集。**

<details>
<summary>参考代码</summary>

```python
from itertools import combinations

def find_all_valid_adjustment_sets(edges, treatment, outcome):
    """
    找出所有满足后门准则的调整集
    
    策略：
    1. 列举所有可能的变量子集
    2. 对每个子集检验是否满足后门准则
    3. 找出最小的（节点数最少）
    """
    # 获取所有节点
    all_nodes = set()
    for u, v in edges:
        all_nodes.add(u)
        all_nodes.add(v)
    
    # 排除 T 和 Y
    candidates = all_nodes - {treatment, outcome}
    
    # 找出 T 的后代（不能控制）
    descendants = find_descendants(edges, treatment)
    candidates = candidates - descendants
    
    # 枚举所有子集，从小到大
    valid_sets = []
    for size in range(len(candidates) + 1):
        for subset in combinations(candidates, size):
            adj_set = set(subset)
            if is_valid_adjustment_set(edges, treatment, outcome, adj_set):
                valid_sets.append(adj_set)
    
    # 找出最小的
    if not valid_sets:
        return []
    
    min_size = min(len(s) for s in valid_sets)
    minimal_sets = [s for s in valid_sets if len(s) == min_size]
    
    return minimal_sets

def find_descendants(edges, node):
    """找出一个节点的所有后代"""
    descendants = set()
    to_visit = [v for (u, v) in edges if u == node]
    
    while to_visit:
        current = to_visit.pop()
        if current not in descendants:
            descendants.add(current)
            to_visit.extend([v for (u, v) in edges if u == current])
    
    return descendants

# 测试
edges = [
    ("X1", "T"), ("X1", "Y"),  # X1 是混淆
    ("X2", "T"), ("X2", "Y"),  # X2 是混淆
    ("T", "M"), ("M", "Y"),     # M 是中介
    ("T", "C"), ("Y", "C")      # C 是碰撞
]

minimal = find_all_valid_adjustment_sets(edges, "T", "Y")
print("最小调整集:")
for s in minimal:
    print(f"  {s if s else '{}'}")

# 输出: {X1, X2}, {X1}, {X2} 都可能是最小调整集
# 具体取决于是否两个都需要阻断所有后门路径
```

**关键点**:
- 必须阻断所有后门路径
- 不能包含 T 的后代（如 M）
- 不应包含碰撞变量（如 C）
- 最小调整集可能不唯一

</details>

---

**Q5: 实现一个函数，模拟三种因果结构（Fork, Chain, Collider），验证控制中间节点的不同效果。**

<details>
<summary>参考代码</summary>

```python
def simulate_three_structures(n=2000, seed=42):
    """
    模拟并对比三种因果结构
    
    1. Fork: T ← X → Y (混淆)
    2. Chain: T → M → Y (中介)
    3. Collider: T → C ← Y (碰撞)
    """
    np.random.seed(seed)
    results = {}
    
    # ===== Structure 1: Fork (混淆) =====
    X = np.random.randn(n)
    T_fork = (X + np.random.randn(n) > 0).astype(int)
    Y_fork = X + np.random.randn(n) * 0.5  # Y 不受 T 影响，只受 X 影响
    
    # 不控制 X: T 和 Y 看起来相关
    corr_unconditional_fork = np.corrcoef(T_fork, Y_fork)[0, 1]
    
    # 控制 X: T 和 Y 独立
    from sklearn.linear_model import LinearRegression
    model = LinearRegression().fit(np.column_stack([T_fork, X]), Y_fork)
    residual_Y = Y_fork - model.predict(np.column_stack([T_fork, X]))
    residual_T = T_fork - LinearRegression().fit(X.reshape(-1, 1), T_fork).predict(X.reshape(-1, 1))
    corr_conditional_fork = np.corrcoef(residual_T, residual_Y)[0, 1]
    
    results['Fork'] = {
        '不控制X': corr_unconditional_fork,
        '控制X': corr_conditional_fork
    }
    
    # ===== Structure 2: Chain (中介) =====
    T_chain = np.random.randn(n)
    M = T_chain + np.random.randn(n) * 0.5
    Y_chain = M + np.random.randn(n) * 0.5  # Y 通过 M 受 T 影响
    
    # 不控制 M: T 和 Y 相关（总效应）
    corr_unconditional_chain = np.corrcoef(T_chain, Y_chain)[0, 1]
    
    # 控制 M: T 和 Y 独立（直接效应=0）
    model = LinearRegression().fit(np.column_stack([T_chain, M]), Y_chain)
    residual_Y = Y_chain - model.predict(np.column_stack([T_chain, M]))
    residual_T = T_chain - LinearRegression().fit(M.reshape(-1, 1), T_chain).predict(M.reshape(-1, 1))
    corr_conditional_chain = np.corrcoef(residual_T, residual_Y)[0, 1]
    
    results['Chain'] = {
        '不控制M': corr_unconditional_chain,
        '控制M': corr_conditional_chain
    }
    
    # ===== Structure 3: Collider (碰撞) =====
    T_collider = np.random.randn(n)
    Y_collider = np.random.randn(n)  # T 和 Y 独立
    C = T_collider + Y_collider + np.random.randn(n) * 0.5
    
    # 不控制 C: T 和 Y 独立
    corr_unconditional_collider = np.corrcoef(T_collider, Y_collider)[0, 1]
    
    # 控制 C: T 和 Y 相关（虚假关联）
    model = LinearRegression().fit(np.column_stack([T_collider, C]), Y_collider)
    residual_Y = Y_collider - model.predict(np.column_stack([T_collider, C]))
    residual_T = T_collider - LinearRegression().fit(C.reshape(-1, 1), T_collider).predict(C.reshape(-1, 1))
    corr_conditional_collider = np.corrcoef(residual_T, residual_Y)[0, 1]
    
    results['Collider'] = {
        '不控制C': corr_unconditional_collider,
        '控制C': corr_conditional_collider
    }
    
    return results

# 运行并展示
results = simulate_three_structures()
for structure, corrs in results.items():
    print(f"\n{structure}:")
    for condition, corr in corrs.items():
        print(f"  {condition}: {corr:.4f}")

# 预期输出:
# Fork: 不控制X 高相关，控制X 低相关 (阻断混淆)
# Chain: 不控制M 高相关，控制M 低相关 (阻断因果链)
# Collider: 不控制C 低相关，控制C 高相关 (打开路径!)
```

**关键发现**:
1. Fork 和 Chain: 控制中间节点**阻断**关联
2. Collider: 控制中间节点**打开**关联（相反！）
3. 这验证了 d-分离的三种规则

</details>

---

---

## 📐 数学推导

### 1. d-分离的形式化定义

**定义**: 在 DAG G 中，给定节点集合 Z，如果 Z 阻断了 X 和 Y 之间的所有路径，则称 X 和 Y 被 Z d-分离。

数学表达：$(X \perp\!\!\!\perp Y \mid Z)_G$

**阻断规则** (Pearl, 1988):

一条路径 P 被集合 Z 阻断，当且仅当：

1. **Fork (叉子)**: $A \leftarrow B \rightarrow C$
   - 如果 B ∈ Z，路径被阻断
   - 直觉：控制中间节点阻断关联

2. **Chain (链条)**: $A \rightarrow B \rightarrow C$
   - 如果 B ∈ Z，路径被阻断
   - 直觉：控制中间节点阻断传递

3. **Collider (碰撞)**: $A \rightarrow B \leftarrow C$
   - 如果 B ∉ Z **且** B 的所有后代都 ∉ Z，路径被阻断
   - 如果 B ∈ Z **或** B 的某个后代 ∈ Z，路径被**打开**
   - 直觉：碰撞默认阻断，控制后反而打开

**d-分离定理** (Pearl):
$$P(X, Y \mid Z) = P(X \mid Z) \cdot P(Y \mid Z) \iff (X \perp\!\!\!\perp Y \mid Z)_G$$

即：图中的 d-分离对应概率分布中的条件独立。

---

### 2. 后门准则的形式化证明

**后门准则** (Pearl, 1993): 

给定 DAG G，变量集合 Z 满足相对于 (T, Y) 的后门准则，当且仅当：

1. Z 中没有 T 的后代
2. Z 阻断所有从 T 到 Y 的包含指向 T 的箭头的路径

**定理**: 如果 Z 满足后门准则，则：

$$P(Y(t)) = \sum_z P(Y \mid T=t, Z=z) \cdot P(Z=z)$$

**证明思路**:

1. **因果效应定义**: 
   $$P(Y(t)) = \sum_x P(Y(t) \mid X=x) \cdot P(X=x)$$
   其中 X 是所有变量。

2. **do-calculus 规则**: 
   如果 Z 阻断所有后门路径，则：
   $$P(Y \mid do(T=t), Z=z) = P(Y \mid T=t, Z=z)$$
   
3. **边际化**: 
   $$P(Y \mid do(T=t)) = \sum_z P(Y \mid do(T=t), Z=z) \cdot P(Z=z)$$
   
4. **替换**: 
   $$= \sum_z P(Y \mid T=t, Z=z) \cdot P(Z=z)$$

**直觉**: 控制 Z 后，T 对 Y 的效应是因果的（不再有混淆）。

---

### 3. 碰撞偏差的数学说明

**设定**: T → C ← Y，T 和 Y 本来独立

**命题**: 在总人口中 $T \perp\!\!\!\perp Y$，但条件于 C 后，T 和 Y 不独立。

**证明**:

假设：
- P(T = 1) = P(T = 0) = 0.5
- P(Y = 1) = P(Y = 0) = 0.5
- P(C = 1 | T, Y) = (T + Y) / 2

即：C = 1 需要 T 或 Y 至少有一个为 1。

**总人口**: 
$$P(T = 1, Y = 1) = P(T = 1) \cdot P(Y = 1) = 0.25$$
$$P(T = 1) \cdot P(Y = 1) = 0.5 \times 0.5 = 0.25$$

因此 $T \perp\!\!\!\perp Y$ ✓

**条件于 C = 1**: 
$$P(T = 1, Y = 1 \mid C = 1) = \frac{P(C = 1 \mid T = 1, Y = 1) \cdot P(T = 1, Y = 1)}{P(C = 1)}$$

计算：
- $P(C = 1 \mid T = 1, Y = 1) = 1$
- $P(C = 1 \mid T = 1, Y = 0) = 0.5$
- $P(C = 1 \mid T = 0, Y = 1) = 0.5$
- $P(C = 1 \mid T = 0, Y = 0) = 0$

$$P(C = 1) = 1 \times 0.25 + 0.5 \times 0.25 + 0.5 \times 0.25 + 0 \times 0.25 = 0.5$$

$$P(T = 1, Y = 1 \mid C = 1) = \frac{1 \times 0.25}{0.5} = 0.5$$

而：
$$P(T = 1 \mid C = 1) \cdot P(Y = 1 \mid C = 1) = 0.5 \times 0.5 = 0.25 \neq 0.5$$

因此 $T \not\perp\!\!\!\perp Y \mid C$ ✗

**结论**: 控制碰撞变量会引入依赖关系（虚假关联）。

---

### 4. 前门准则（额外知识）

当存在未观测混淆 U 但有中介变量 M 时：

**DAG**: $T \rightarrow M \rightarrow Y, U \rightarrow T, U \rightarrow Y$

**前门准则**: 如果：
1. M 阻断所有从 T 到 Y 的有向路径
2. 不存在从 T 到 M 的后门路径
3. T 阻断所有从 M 到 Y 的后门路径

则：
$$P(Y(t)) = \sum_m P(M = m \mid T = t) \sum_{t'} P(Y \mid M = m, T = t') \cdot P(T = t')$$

**应用**: 当无法控制混淆 U，但可以测量中介 M 时使用。

---

In [ ]:
# 可视化碰撞偏差
if overall is not None:
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # 图1: 总人口
    ax1 = axes[0]
    stars = df_collider[df_collider['明星'] == 1]
    non_stars = df_collider[df_collider['明星'] == 0]
    
    ax1.scatter(non_stars['演技'], non_stars['颜值'], 
                alpha=0.3, c='gray', label='普通人', s=20)
    ax1.scatter(stars['演技'], stars['颜值'], 
                alpha=0.7, c='gold', label='明星', s=50, edgecolors='black')
    ax1.set_xlabel('演技')
    ax1.set_ylabel('颜值')
    ax1.set_title(f'总人口\n相关性 = {overall:.3f}')
    ax1.legend()
    
    # 图2: 只看明星
    ax2 = axes[1]
    ax2.scatter(stars['演技'], stars['颜值'], 
                alpha=0.7, c='gold', s=50, edgecolors='black')
    
    # 添加趋势线
    z = np.polyfit(stars['演技'], stars['颜值'], 1)
    p = np.poly1d(z)
    x_line = np.linspace(stars['演技'].min(), stars['演技'].max(), 100)
    ax2.plot(x_line, p(x_line), 'r--', linewidth=2, label='趋势线')
    
    ax2.set_xlabel('演技')
    ax2.set_ylabel('颜值')
    ax2.set_title(f'只看明星群体\n相关性 = {conditional:.3f} (虚假负相关！)')
    ax2.legend()
    
    plt.tight_layout()
    plt.show()
    
    print("\n💡 启示: 控制碰撞变量会创造虚假关联！")
    print("   在因果推断中，绝对不能控制碰撞变量。")

---

## 📝 Part 7: 思考题

### 问题 1: DAG 分析

给定以下 DAG:

```
X → T → Y ← U → X
```

回答:
- T 和 Y 之间有哪些路径？
- 哪些是后门路径？
- 最小调整集是什么？

**你的答案:**

*（在这里写下你的分析...）*

---

### 问题 2: 为什么控制碰撞变量会引入偏差？

**提示:** 想想好莱坞悖论的例子...

**你的答案:**

*（在这里写下你的思考...）*

---

### 问题 3: 中介变量应该控制吗？

在以下 DAG 中：T → M → Y

- 什么时候应该控制 M？
- 什么时候不应该控制 M？

**你的答案:**

*（在这里写下你的思考...）*

---

### 问题 4: 未观测混淆

如果存在未观测的混淆变量 U（我们没有数据），有什么方法可以识别因果效应？

**提示:** 想想工具变量、断点回归、双重差分...

**你的答案:**

*（在这里写下你的思考...）*

---

---

## 🎉 总结

### 核心知识点

| 概念 | 定义 | 规则 |
|-----|------|------|
| 混淆变量 | T ← X → Y | 必须控制 |
| 中介变量 | T → M → Y | 估计总效应时不控制 |
| 碰撞变量 | T → C ← Y | 绝对不能控制 |

### 后门准则

要无偏估计 T 对 Y 的效应：
1. 找出所有后门路径
2. 选择一组变量阻断所有后门路径
3. 确保这组变量不包含 T 的后代

### 下一步

现在我们知道了「应该控制什么」，下一个练习我们将深入学习**混淆偏差**——它有多大？怎么量化？怎么避免？

---

**「画图先行，分析后行」是因果推断的第一原则。**